# Exercise 3.3. Movement of a robot using odometry commands

## Theoretical background

The *odometry motion model* is more suitable to keep track and estimate the robots pose in contrast to the *velocity model*. The reason being, most robot bases provide some form of *odometry information*, a measurement of how much the robot has moved in reality, whose greater precision makes it useful to keep track of the pose.

Although technically it is a measurement rather than a control, we 
treat it as control to simplify the modeling. The odometry commands take the form of:

$$
    u_t = \begin{bmatrix}
            \Delta x  \\
            \Delta y \\
            \Delta \theta
        \end{bmatrix}
$$

We'll implement this motion model in both analytical and sample form.

In [ ]:
%matplotlib notebook
#%matplotlib inline

# IMPORTS

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from utils.PlotEllipse import PlotEllipse
from utils.DrawRobot import DrawRobot
from utils.tcomp import tcomp
from utils.pause import pause
from utils.Jacobians import J1, J2

## 3.3.1 Analytical form

**Context**

Just as we did in lesson 3.1, the analytic form of the odometry motion model uses the composition of poses to model the robot's movement, providing only a notion of how much the pose has changed, not how did it get there.

$$
    p_1 \oplus \Delta p
    = \begin{bmatrix}
        x_1 + \Delta x \cos \theta_1 - \Delta y \sin \theta_1 \\ 
        y_1 + \Delta x \sin \theta_1 - \Delta y \cos \theta_1 \\
        \theta_1 + \Delta \theta
      \end{bmatrix}
$$

The crux of this assignment is to keep track of the covariance matrix of the pose $(P)$, using the Jacobians of this movement model.

**Assignment**

Similarly to the exercise 3.1, we'll move a robot along a 8-by-8 square (in meters), in increments of 2m.
Just as the exercise 3.2.2, you'll need to draw the uncertainty of the pose(as an ellipse) and the ground-truth of the position(randomly generated).

$$
    \Sigma_{\Delta_p} = \begin{bmatrix}
        0.04 & 0 & 0 \\
        0 & 0.04 & 0 \\
        0 & 0 & 0.01
    \end{bmatrix}
$$

**Example**

<figure style="text-align:center">
  <img src="images/fig3-3-1.png" width="400" alt="">
  <figcaption>Fig. 1: Movement of a robot using odometry commands. <br/> Representing the expected pose (in red), the true pose (as dots) <br/> and the confidence ellipse.</figcaption>
</figure>

In [ ]:
class Robot():
    """ Simulation of a robot base
    
        Attrs:
            pose: Expected pose of the robot
            P: Covariance of the current pose
            true_pose: Real pose of the robot(affected by noise)
            Q: Covariance of the movement
    """
    def __init__(self, x, P, Q):
        self.pose = x
        self.P = P
        self.true_pose = self.pose
        self.Q = Q
        
    def step(self, u):
        # TODO Update expected pose
        self.pose = None
        
        # TODO Generate true pose 
        self.true_pose = None
        
        # TODO Update covariance
        self.P = None
        
        raise NotImplementedError
    
    def draw(self, fig, ax):
        DrawRobot(fig, ax, self.pose)
        el = PlotEllipse(fig, ax, self.pose, self.P)
        ax.plot(self.true_pose[0, 0], self.true_pose[1, 0], 'o', color=el[0].get_color())
    
def demo1(robot):  
    # MATPLOTLIB
    fig, ax = plt.subplots()
    ax.set_xlim([-3, 11])
    ax.set_ylim([-3, 11])
    plt.ion()
    plt.grid()
    plt.tight_layout()
    fig.canvas.draw()
    
    # MOVEMENT PARAMETERS
    nSteps = 15
    ang = -np.pi/2 # angle to turn in corners
    u = np.vstack((2., 0., 0.))
    
    # MAIN LOOP
    for i in range(nSteps):
        # change angle on corners
        if i % 4 == 3:
            u[2, 0] = ang

        #Update positions
        robot.step(u)

        # Restore angle iff changed
        if i % 4 == 3:
            u[2, 0] = 0

        # Draw every loop
        robot.draw(fig, ax)
        fig.canvas.draw()
        plt.pause(0.3)

In [ ]:
x = np.vstack([0., 0., np.pi/2]) # pose inicial

# Probabilistic parameters
P = np.diag([0., 0., 0.])
Q = np.diag([0.04, 0.04, 0.01])

robot = Robot(x, P, Q)
demo1(robot)

## 3.3.2 Sample form

**Context**

The analytical form used above, although useful for the probabilistic algorithms we will cover in this course, does not work well for sampling algorithms such as particle filters.

The reason being, if we generate random samples from the gaussian distributions generated in the previous exercise, we will find some poses that are not feasible to the non-holonomic movement of a robot, i.e. they do not correspond to a velocity command $(v, w)$ with noise.

The following *sample form* is a more realistic way to generate samples of the pose. Now we model the movement of the robot as a sequence of actions: 

1. **Turn**: to face the destination point.
2. **Advance**: to arrive at the destination.
3. **Turn**: to get to the desired angle.

This type of order is expressed as:

$$
    u_t = \begin{bmatrix}
            \theta_1  \\
            d \\
            \theta_2
        \end{bmatrix}
$$

It can easily be generated from odometry poses $[\hat x_t, \hat y_t,\hat \theta_t]'$ and $[\hat x_{t-1}, \hat y_{t-1},\hat \theta_{t-1}]'$ given the following equations:

$$
    \begin{equation}
    \theta_1 =atan2(\hat y_t -\hat y_{t-1}, \hat x_t -\hat x_{t-1})- \hat \theta_{t-1} \\
    d = \sqrt{(\hat y_t -\hat y_{t-1})^2 + (\hat x_t -\hat x_{t-1})^2} \\
    \theta_2  = \hat{\theta}_t - \hat{\theta}_{t-1} - \theta_1
    \end{equation}
$$

**Assignment**

1. Implement a function that, given the previously mentioned $[\hat x_t, \hat y_t,\hat \theta_t]'$ and $[\hat x_{t-1}, \hat y_{t-1},\hat \theta_{t-1}]'$ generates an order $u_t = [ \theta_1, d , \theta_2 ]'$

  Expected output for the commented example:

  ```
  array([[-3.92699082],
       [ 1.41421356],
       [ 2.35619449]])
  ```

In [ ]:
def generate_move(pose_now, pose_old):
    # TODO Complete the function
    raise NotImplementedError
    
# EXAMPLE
#generate_move(np.vstack([0., 0., 0.]), np.vstack([1., 1., np.pi/2]))

2. Using the resulting control action $u_t = [\theta_1, d, \theta_2]'$ we can model the noise of the action in the following way:

  $$
    \begin{equation}
        \theta_1 = \hat\theta_1 + \text{sample}\left(\alpha_0 \hat\theta_1^2 + \alpha_1 \hat d^2 \right) \\
        d = \hat d + \text{sample}\left(\alpha_2 \hat d^2 + \alpha_3 \left(\hat\theta_1^2 + \hat d^2 \right) \right) \\
        \theta_2 = \hat\theta_2 + \text{sample}\left(\alpha_0 \hat\theta_2^2 + \alpha_1 \hat d^2\right)
    \end{equation}
  $$

  Where $sample(b)$ generates a random value from a distribution $N(0, b)$. The vector $\alpha = [\alpha_0, \dots, \alpha_3]$, models the robot's noise.

  The pose of the robot at the end of the movement is computed as follows:

$$
    \begin{equation}
        x_t = x_{t-1} + d \cos\left(\theta_{t-1} + \theta_1 \right) \\
        y_t = y_{t-1} + d \sin\left(\theta_{t-1} + \theta_1 \right) \\
        \theta_t = \theta_{t-1} +  \theta_1 +  \theta_2
    \end{equation}
$$

In [ ]:
class SampledRobot(object):
    def __init__(self, mean, a, n_samples):
        self.pose = mean
        self.a = a
        self.samples = np.tile(mean, n_samples)
        
    def step(self, u):
        # TODO Update pose
        self.pose = None
        
        # TODO Generate new samples
        self.samples = None
        
        raise NotImplementedError
        
    def draw(self, fig, ax):
        DrawRobot(fig, ax, self.pose)
        ax.plot(self.samples[0, :], self.samples[1, :], '.')   

Draw the pose of the robot (without angle) as a cloud of particles (samples of possible points which the robot can be at). Use the variables in the code below. Play with different values of 'a'. To improve this visualization the robot will move in increments of $0.5$ and we are to plot the particles each 4 increments.

**Example**

<figure style="text-align:center">
  <img src="images/fig3-3-2.png" width="400" alt="">
  <figcaption>Fig. 1: Movement of a robot using odometry commands in sampling form. <br/> Representing the expected pose (in red) and the samples (as clouds of dots) </figcaption>
</figure>

In [ ]:
def demo2(robot):
    # PARAMETERS
    inc = .5
    show_each = 4
    limit_iterations = 32
    
    # MATPLOTLIB
    fig, ax = plt.subplots()
    ax.set_xlim([-3, 11])
    ax.set_ylim([-3, 11])
    plt.ion()
    plt.grid()
    plt.tight_layout()
    
    # MAIN LOOP
    robot.draw(fig, ax)
    inc_pose = np.vstack((0., inc, 0.))
    
    for i in range(limit_iterations):
        if i == 16:
            inc_pose[0, 0] = inc
            inc_pose[1, 0] = 0
            inc_pose[2, 0] = -np.pi/2
            
        u = generate_move(robot.pose+inc_pose, robot.pose)
        
        robot.step(u)
        
        if i == 16:
            inc_pose[2, 0] = 0
 
        if i % show_each == show_each-1:
            robot.draw(fig, ax)
            fig.canvas.draw()
            plt.pause(0.1)

In [ ]:
n_particles = 100
a = np.array([.07, .07, .03, .05])
x = np.vstack((0., 0., np.pi/2))

robot = SampledRobot(x, a, n_particles)
demo2(robot)
plt.close()